In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import classification_report
import pickle
import mlflow
import dagshub

In [5]:
# Load and prepare data
df = pd.read_csv(r'C:\Users\dell\Downloads\diabetes.csv')
#df = pd.read_csv(data)
X = df.drop('Outcome', axis=1)
y = df.Outcome

In [6]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [8]:
# Handle missing values
from sklearn.impute import SimpleImputer
fill = SimpleImputer(missing_values=0, strategy="mean")
X_train = fill.fit_transform(X_train)
X_test = fill.transform(X_test)

In [10]:
# Initialize DagsHub
dagshub.init(repo_owner='raguram99', repo_name='mlops_projects', mlflow=True)

Initialized MLflow to track repo "raguram99/mlops_projects"

Repository raguram99/mlops_projects initialized!

In [12]:
# Define models and their parameters
models = {
    'logistic_regression': {
        'model': LogisticRegression(),
        'params': {
            "solver": "lbfgs",
            "max_iter": 45,
            "multi_class": "auto",
            "random_state": 123
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': 100,
            'max_depth': 10,
            'random_state': 123
        }
    },
    'xgboost': {
        'model': xgb.XGBClassifier(),
        'params': {
            'n_estimators': 100,
            'max_depth': 3,
            'learning_rate': 0.1,
            'random_state': 123
        }
    },
    'svm': {
        'model': SVC(),
        'params': {
            'kernel': 'rbf',
            'C': 1.0,
            'random_state': 123
        }
    }
}


In [13]:
# Train and log each model
mlflow.set_experiment("Multi_Classifier_Diabetes_Experiment")

for model_name, model_info in models.items():
    print(f"\nTraining {model_name}...")
    
    with mlflow.start_run(run_name=model_name):
        # Set tags
        mlflow.set_tag("author", "AJ")
        mlflow.set_tag("model_type", model_name)
        
        # Train model
        model = model_info['model']
        model.set_params(**model_info['params'])
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Get metrics
        report_dict = classification_report(y_test, y_pred, output_dict=True)
        
        # Log parameters
        mlflow.log_params(model_info['params'])
        
        # Log metrics
        metrics = {
            'accuracy': report_dict['accuracy'],
            'recall_class_0': report_dict['0']['recall'],
            'recall_class_1': report_dict['1']['recall'],
            'f1_score_macro': report_dict['macro avg']['f1-score']
        }
        mlflow.log_metrics(metrics)
        
        # Save and log model
        filename = f'{model_name}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        mlflow.log_artifact(filename, model_name)
        
        print(f"{model_name} Results:")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score (macro): {metrics['f1_score_macro']:.4f}")

print("\nAll models have been trained and logged to MLflow")

2025/09/14 14:52:40 INFO mlflow.tracking.fluent: Experiment with name 'Multi_Classifier_Diabetes_Experiment' does not exist. Creating a new experiment.



Training logistic_regression...


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 45 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=45).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logistic_regression Results:
Accuracy: 0.6623
F1 Score (macro): 0.5970
🏃 View run logistic_regression at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0/runs/a425b05f1549488e802a2e97e4ec10e7
🧪 View experiment at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0

Training random_forest...
random_forest Results:
Accuracy: 0.7208
F1 Score (macro): 0.6893
🏃 View run random_forest at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0/runs/ef35f5c6836145d1a8d78d3843031258
🧪 View experiment at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0

Training xgboost...
xgboost Results:
Accuracy: 0.7403
F1 Score (macro): 0.7172
🏃 View run xgboost at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0/runs/40706ad115ca477f8d386db487627285
🧪 View experiment at: https://dagshub.com/raguram99/mlops_projects.mlflow/#/experiments/0

Training svm...
svm Results:
Accuracy: 0.7013
F1 Score (macro): 0.6435
🏃 View run